In [2]:
from abc import ABC,abstractmethod

class NotificationSender(ABC):
    @abstractmethod
    def send(self,messege) ->str: ...

class EmailNotification(NotificationSender):
    def send(self,messege):
        return f"[email] sending: {messege}"
    
class SMSNotification(NotificationSender):
    def send(self,messege):
        return f"[sms] sending: {messege}"
    
class Notification(ABC):
    def __init__(self,sender) -> None:
        self.sender = sender

    def notify(self,messege:str): ...
        
class AlertNotification(Notification):
    def notify(self,messege:str):
        return self.sender.send(messege)

email = EmailNotification()
sms = SMSNotification()

alert = AlertNotification(email)
print(alert.notify("система перегружена"))

alert.sender = sms
print(alert.notify("низкий заряд батареи"))

[email] sending: система перегружена
[sms] sending: низкий заряд батареи


In [3]:
class PushNotification:
    def push(self,content:str):
        return f"[push] sending: {content}"

class PushNotificationAdapter(NotificationSender):
    def __init__(self,adaptee:PushNotification) -> None:
        self.adaptee = adaptee

    def send(self,messege) ->str:
        return self.adaptee.push(messege)
    

push = PushNotification()
adapter = PushNotificationAdapter(push)

alert = AlertNotification(adapter)
print(alert.notify("новый способ сообщения"))


[push] sending: новый способ сообщения


In [7]:
class BaseNotification(ABC):
    @abstractmethod
    def notify(self,messege) ->str: ...

class SimpleNotification(BaseNotification):
    def __init__(self,sender) ->None:
        self.sender = sender

    def notify(self,messege) -> str:
        return self.sender.send(messege)
    
class UrgentNotification(BaseNotification):
    def __init__(self,wrapped: BaseNotification) ->None:
        self.wrapped = wrapped

    def notify(self,messege) -> str:
        return "[urgent]"+""+ self.wrapped.notify(messege)
    
email = EmailNotification()
simple = SimpleNotification(email)

urgent = UrgentNotification(simple)
urgent.notify("сервер упал")


'[urgent][email] sending: сервер упал'

In [2]:
class WaterHeater:
    def heat(self):
        print("нагрев воды")

class CoffeeGrinder:
    def grind(self):
        print("молем кофе")

class CoffeeBreaker:
    def brew(self):
        print("варим кофе")

class CoffeeMachine:
    def __init__(self) -> None:
        self.header = WaterHeater()
        self.grider = CoffeeGrinder()
        self.brewer = CoffeeBreaker()

    def make_coffee(self):
        print("-начинаем готовит кофе: ")
        self.header.heat()
        self.grider.grind()
        self.brewer.brew()
        print("кофе готов")


machine = CoffeeMachine()
machine.make_coffee()

-начинаем готовит кофе: 
нагрев воды
молем кофе
варим кофе
кофе готов


In [7]:
class TreeType:
    def __init__(self,name,color,texture) ->None:
        self.name = name
        self.color = color
        self.texture = texture

    def draw(self,x,y):
        print(f"рисуем {self.name} {self.color} в точке x:{x}, y:{y}")

class TreeFactory:
    _tree_types = {}

    @classmethod
    def get_tree_type(cls,name,color,texture):
        key = (name,color,texture)
        if key not in cls._tree_types:
            cls._tree_types[key] = TreeType(name,color,texture)
        return cls._tree_types[key]

class Tree:
    def __init__(self,x,y,tree_type:TreeType) -> None:
        self.x = x
        self.y = y
        self.type = tree_type

    def draw(self):
        self.type.draw(self.x,self.y)

pine_type = TreeFactory.get_tree_type("сосна","зеленый","хвоя.png")

tree1 = Tree(10,20,pine_type)
tree2 = Tree(30,40,pine_type)

tree1.draw()
tree2.draw()

рисуем сосна зеленый в точке x:10, y:20
рисуем сосна зеленый в точке x:30, y:40


In [9]:
class RealDatabase:
    def connect(self):
        print("подключение к базе данных")

    def query(self,sql):
        print(f"выполнение запроса: {sql}")

class DatabaseProxy:
    def __init__(self):
        self.real_bd = None
        self.connected= False

    def connect(self):
        if not self.connected:
            self.real_bd = RealDatabase()
            self.real_bd.connect()
            self.connected = True
        else:
            print("уже подключено")

    def query(self,sql):
        self.connect()
        self.real_bd.query(sql)

db = DatabaseProxy()
db.query("SELECT * FROM users")
db.query("SELECT * FROM products")


подключение к базе данных
выполнение запроса: SELECT * FROM users
уже подключено
выполнение запроса: SELECT * FROM products


In [13]:
from abc import ABC,abstractmethod

class Handler(ABC):
    def __init__(self) -> None:
        self.next_handler = None

    def set_next(self,handler):
        self.next_handler = handler
        return handler
    
    @abstractmethod
    def handle(self,request): ...

class Level1Support(Handler):
    def handle(self, request):
        if request == "пароль":
            print("level 1: помогаем восстановить пароль")

        elif self.next_handler:
            self.next_handler.handle(request)

class Level2Support(Handler):
    def handle(self, request):
        if request == "ошибка программы":
            print("level 2: решает ошибку")

        elif self.next_handler:
            self.next_handler.handle(request)

class Level3Support(Handler):
    def handle(self, request):
        print(f"level 3: эскалация запроса: {request}")

level1 = Level1Support()
level2 = Level2Support()
level3 = Level3Support()

level1.set_next(level2).set_next(level3)

level1.handle("пароль")
level1.handle("ошибка программы")
level1.handle("запрос денег")

level 1: помогаем восстановить пароль
level 2: решает ошибку
level 3: эскалация запроса: запрос денег


In [17]:
from abc import ABC,abstractmethod

class Light:
    def turn_on(self):
        print("свет включен")

    def turn_off(self):
        print("свет выключен")

class Command(ABC):
    @abstractmethod
    def execute(self): ...

class LightOnCommand(Command):
    def __init__(self,light:Light) -> None:
        self.light = light

    def execute(self):
        self.light.turn_on()

class LightOffCommand(Command):
    def __init__(self,light:Light) -> None:
        self.light = light

    def execute(self):
        self.light.turn_off()

class RemoteControl:
    def __init__(self) -> None:
        self.command = None

    def set_command(self,command: Command):
        self.command = command

    def press_button(self):
        if self.command:
            self.command.execute()

light = Light()

on = LightOnCommand(light)
off = LightOffCommand(light)

remote = RemoteControl()
remote.set_command(on)
remote.press_button()

remote.set_command(off)
remote.press_button()

свет включен
свет выключен


In [ ]:
class User:
    def __init__(self,user_id,user_type) -> None:
        self.user_id = user_id
        self.user_type = user_type

    def has_permission(self,permission):
        return self.user_type in permission
    
class BookFlyweight:
    def __init__(self,title,author) -> None:
        self.title = title
        self.author = author
        self.issued = False

    def is_issued(self):
        return self.issued
    
    def issue(self):
        self.issued = True

    def return_book(self):
        self.issued = False

class BookProxy:
    def __init__(self,books):
        self.books = books

    def add_book(self,book_id,title,author):
        if book_id not in self.books:
            self.books[book_id] = BookFlyweight(title,author)

    def check_availability(self,book_id):
        return book_id in self.books and not self.books[book_id].is_issued()

    def issue_book(self,book_id,user_id):
        if self.check_availability(book_id):
            print(f"Book {book_id} выдана пользователю {user_id}")

        else:
            print(f"book {book_id} не доступна")
    
    def return_book(self,book_id,user_id):
        print(f"book {book_id} была возвращена пользователем {user_id}")


class LibraryFacade:
    def __init__(self) -> None:
        self.users = {}
        self.books = {}
        self.book_proxy = BookProxy(self.books)

    def add_book(self,book_id,title,author):
        self.book_proxy.add_book(book_id,title,author)

    def issue_book(self,book_id,user_id):
        self.book_proxy.issue_book(book_id,user_id)

    def return_book(self,book_id,user_id):
        self.book_proxy.return_book(book_id,user_id)

    def check_book_availability(self,book_id):
        return self.book_proxy.check_availability(book_id)

library = LibraryFacade()

library.add_book(1,"title 1","author 1")
library.add_book(2,"title 2","author 2")

user1 = User(1,"reader")
user2 = User(2,"librian")

print(library.check_book_availability(1))

library.issue_book(1,user1.user_id)
library.return_book(1,user1.user_id)

True
Book 1 выдана пользователю 1
book 1 была возвращена пользователем 1


In [3]:
class Myiterator:
    def __init__(self,items) -> None:
        self.items = items
        self.index = 0

    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.index < len(self.items):
            item = self.items[self.index]
            self.index += 1
            return item
        else:
            print("итерация кончилась!")
            raise StopIteration


class MyCollection:
    def __init__(self,items) -> None:
        self.items = items

    def __iter__(self):
        return Myiterator(self.items)
    
collection = MyCollection([1,2,3,4,5])

for item in collection:
    print(item)


1
2
3
4
5
итерация кончилась!


In [5]:
class User:
    def __init__(self,name) -> None:
        self.name = name
        self.mediator = None

    def set_mediator(self,mediator):
        self.mediator = mediator

    def send_messege(self,messege):
        print(f"{self.name} отправил сообщение: {messege}")
        self.mediator.send_messege(self,messege)

    def receive_messege(self,messege):
        print(f"{self.name} получил сообщение: {messege}")

class ChatMediator:
    def __init__(self) -> None:
        self.users = []

    def add_user(self,user):
        self.users.append(user)
        user.set_mediator(self)

    def send_messege(self,sender,messege):
        for user in self.users:
            if user != sender:
                user.receive_messege(messege)


chat_mediator = ChatMediator()

alice = User("Alice")
bob = User("Bob")
jhon = User("Jhon")

chat_mediator.add_user(alice)
chat_mediator.add_user(bob)
chat_mediator.add_user(jhon)

alice.send_messege("hello")

Alice отправил сообщение: hello
Bob получил сообщение: hello
Jhon получил сообщение: hello


In [7]:
class Memento:
    def __init__(self,text) -> None:
        self.__text = text
    
    def get_text(self):
        return self.__text
    
class TextEditor:
    def __init__(self,text="") -> None:
        self.__text = text

    def set_text(self,text):
        self.__text = text

    def get_text(self):
        return self.__text
    
    def creat_memento(self):
        return Memento(self.__text)
    
    def restore_memento(self,memento):
        self.__text = memento.get_text()


class Caretaker:
    def __init__(self) -> None:
        self.__mementos = []

    def add_memento(self,memento):
        self.__mementos.append(memento)

    def get_memento(self,index):
        return self.__mementos[index]
    
editor = TextEditor()

editor.set_text("hello world!")
print(f"текущий текст: {editor.get_text()}")

caretaker = Caretaker()
caretaker.add_memento(editor.creat_memento())

editor.set_text("hello python")
print(f"текущий текст: {editor.get_text()}")

editor.restore_memento(caretaker.get_memento(0))
print(f"текст после востановления: {editor.get_text()}")

текущий текст: hello world!
текущий текст: hello python
текст после востановления: hello world!
